##### 1. copy conversion functions:
###### - interpolated blm calibration curve and r5im, with integration:
###### - - volts to protons
###### - - coulombs to joules
##### 2. attributes for functions
##### 3. ready to modulate
##### 4. replace integer values with variables
##### 5. work on returns with hen

In [239]:
import glob
import math
import pandas as pd
import paho.mqtt.client as mqtt
import time
import string
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import numpy as np
from scipy import integrate
from scipy.constants import c, m_p, e
import DataFilterModule as dfm

##### csv - make into module

In [240]:
class ImportData:
    def __init__(self):
        index = 0
        folder = None
        cycle = True
    
    def fetch(self, index=0, folder=None, cycle=True):
        if folder is None:
            if cycle:
                folder = './BLM_R5IM_Data/cycle/'
            else:
                folder = './BLM_R5IM_Data/R5IM_loss/'
    
        if folder[-1] != '/': folder += '/'        
        
        input_data = pd.read_csv(glob.glob(folder + '*.csv')[index])
        return input_data.drop(columns = input_data.columns[0]).to_numpy()
    
# csv = ImportData()
# y = csv.fetch()

##### mqtt - make into module

In [241]:
def generate_shortuuid() -> str:
    """Public function for generating short UUID messages"""
    alphabet = string.ascii_lowercase + string.ascii_uppercase + string.digits
    shortuiid = "".join(random.choices(alphabet, k=12))
    return shortuiid

class MQTTClient(mqtt.Client):
    def __init__(self, topic_name, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.topic_name = topic_name

    def on_connect(self, client, userdata, flags, rc):
        print('Connected to: ' + str(self.topic_name))
        self.subscribe(self.topic_name)

    def on_disconnect(self, client, userdata, rc):
        print('Disconnected from: ' + str(self.topic_name))
        
    def on_message(self, client, userdata, message):
        msg_byte = message.payload
        msg_decode = np.frombuffer(msg_byte, dtype=float, count=-1, offset=0)
        msg_array = np.reshape(msg_decode, (40, 2200))
        print(msg_array)

def get_live_data(topic_name, time_period):
    client = MQTTClient(topic_name)
    client.connect("130.246.57.45", 8883, 60)
    client.loop_start()
    time.sleep(time_period)
    client.disconnect()
    client.loop_stop()
        
# get_live_data("ac_phys/workxp/live_signals", 5)

##### filtering - make into module

In [242]:
class DataFilter:
  '''A object with configurable settings to filter input data.
  '''

  labels = ["r0blm1", "r0blm3", "r0blm4",
            "r1blm1", "r1blm2", "r1blm3", "r1blm4",
            "r2blm1", "r2blm2", "r2blm3", "r2blm4",
            "r3blm1", "r3blm2", "r3blm3", "r3blm4",
            "r4blm1", "r4blm2", "r4blm3", "r4blm4",
            "r5blm1", "r5blm2", "r5blm3", "r5blm4",
            "r6blm1", "r6blm2", "r6blm3", "r6blm4",
            "r7blm1", "r7blm2", "r7blm3", "r7blm4",
            "r8blm1", "r8blm2", "r8blm3", "r8blm4",
            "r9blm1", "r9blm2", "r9blm3", "r9blm4",
            "r5im"]
  
  
  def __init__(self, *, select = True, invert = False, scale = 1, offset = 0, auto_offset = 0):
    '''Creates a filterer object, with configurable settings to filter input data.
    '''

    self.data = []

    template = {
      "select": select,
      "invert": invert,
      "scale": scale,
      "offset": offset,
      "auto_offset": auto_offset,
    }

    self.settings = [deepcopy(template) for i in range(40)]

  def _index_(self, label) -> int:
    '''A minor inner method to convert the string name of a BLM to its corresponding index.'''

    if isinstance(label, int):
      return label
    elif isinstance(label, str):
      return DataFilter.labels.index(label)
    else:
      raise TypeError

  
  def invert(self, data: np.array) -> np.array:
    out = -data
    return out

  def scale(self, data: np.array, factor: int | float) -> np.array:
    out = data * factor
    return out
  
  def offset(self, data: np.array, *, offset: int | float = 0, points: int = None) -> np.array:
    '''Offsets data by either `offset`, or zeroes it using a given number of data points as reference for the zero-level.'''
    
    out = data

    if isinstance(points, int):
      if points > 0:
        auto_offset = np.mean(data[:points])
      elif points < 0:
        auto_offset = np.mean(data[points:])
      else:
        raise ValueError("Invalid value for number of reference points")
      out = data - auto_offset

    if offset != 0:
      out = data + offset
    
    return out

  
  def set(self, setting: str, state, labels: list[str | int] = None) -> None:
    '''Configures a particular filter `setting` for a number of BLMs.

    `setting`: the filter setting to configure.
    `state`: the value to set the setting to.
    `labels`: the BLMs to apply the setting to.
    '''

    # if none specified, set for all
    if labels is None:
      for each in self.settings:
        each[setting] = state
    
    else:
      # single specified
      if isinstance(labels, str) or isinstance(labels, int):
        self.settings[self._index_(labels)][setting] = state

      # multiple specified
      else:
        for each in labels:
          self.settings[self._index_(each)][setting] = state

  
  def reset(self) -> None:
    '''Resets filters to their default settings.'''

    self.settings = [{
      "select": True,
      "invert": False,
      "scale": 1,
      "offset": 0,
      "auto_offset": 0
    }] * 40

  
  def apply(self, data: np.array) -> np.array:
    '''Filters `data` according to the configured settings.'''
    
    self.data = data
  
    for i, each in enumerate(self.settings):
      # skip if not selected
      if not each["select"]:
        continue

      self.data[i] = data[i]

      if each["invert"]:
        self.data[i] = self.invert(self.data[i])

      if each["scale"] != 1:
        self.data[i] = self.scale(self.data[i], each["scale"])

      if each["offset"] != 0:
        self.data[i] = self.offset(self.data[i], offset = each["offset"])

      if each["auto_offset"] != 0:
        self.data[i] = self.offset(self.data[i], points = each["auto_offset"])

    return [each for i, each in enumerate(self.data) if self.settings[i]["select"]]

# filterer = dfm.DataFilter()
# filterer.set("invert", True, "r5im")
# filterer.apply(y)

In [243]:
# filterer = dfm.DataFilter()

# filterer.set("invert", True, "r5im")
# filterer.apply(y)

In [244]:
# filterer2 = dfm.DataFilter()

# filterer2.set("auto_offset", 10)
# filterer2.apply(y)

##### graph is:
- x : time interval, blms summed
- y : loss signal
- z : time (rpm)

In [ ]:
def synchrotron_momentum(max_E, time):
    mpeV = m_p * c**2 / e           # Proton mass in eV
    R0 = 26                         # Mean machine radius
    n_dip = 10                      # Number of dipoles
    dip_l = 4.4                     # Dipole length
    
    dip_angle = 2 * np.pi / n_dip   # Dipole bending angle
    rho = dip_l / dip_angle         # Dipole radius of curvature
    omega = 2 * np.pi * 50   
    
    Ek = np.array([70, max_E]) * 1e6 # Injection and extraction kinetic energies 
    E = Ek + mpeV                    # Injection and extraction kinetic energies
    p = np.sqrt(E**2 - mpeV**2)      # Injection and extraction momenta

    B = p / c / rho                  # Ideal magnetic field at injection and extraction energies
    
    Bdip = lambda t: (B[1] + B[0] - (B[1] - B[0]) * np.cos(omega * t)) / 2  # Idealised B-field variation with AC
    pdip = lambda t: Bdip(t) * rho * c                                      # Momentum from B-field in MeV
    
    return pdip(time*1E-3)

def synchrotron_kinetic_energy(max_E, time):
    mpeV = m_p * c**2 / e           # Proton mass in eV    
    # Relativistic Kinetic Energy = Relativistic Energy - mass
    return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV) # Return array in eV
    #return (np.sqrt(synchrotron_momentum(max_E, time)**2 + mpeV**2) - mpeV)/1E6 # Return array in MeV

In [250]:
# calibration curve

BLM_Cal_x_time = np.array([0., 3., 5., 7., 9.])
BLM_Cal_x_energy_ev = synchrotron_kinetic_energy(800., BLM_Cal_x_time)
BLM_Cal_x_energy_MeV = BLM_Cal_x_energy_ev/1E6
BLM_Cal_y = np.array([2.22E-16, 2.59E-16, 4.31E-15, 1.60E-14, 3.50E-14])

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"BLM Calibration Factor $K_{BLM}$ $(\frac{mVs}{proton})$")

ax1.plot(BLM_Cal_x_time,BLM_Cal_y, marker='X')
ax1.set_xlim(-0.5, 10.5)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Initial Calibration Data');

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"BLM Calibration Factor $K_{BLM}$ $(\frac{Vs}{proton})$")

ax1.scatter(BLM_Cal_x_time,BLM_Cal_y, marker='X', c='r', label='Data')
ax1.plot(np.linspace(-0.5, 10.5, 2200), np.interp(np.linspace(-0.5, 10.5, 2200), BLM_Cal_x_time, BLM_Cal_y))

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Linear Interpolation with np.interp');

def divided_diff(x, y):
    '''
    function to calculate the divided
    differences table
    '''
    n = len(y)
    coef = np.zeros([n, n])
    # the first column is y
    coef[:,0] = y
    
    for j in range(1,n):
        for i in range(n-j):
            coef[i][j] = \
           (coef[i+1][j-1] - coef[i][j-1]) / (x[i+j]-x[i])
            
    return coef

def newton_poly(coef, x_data, x):
    '''
    evaluate the newton polynomial 
    at x
    '''
    n = len(x_data) - 1 
    p = coef[n]
    for k in range(1,n+1):
        p = coef[n-k] + (x -x_data[n-k])*p
    return p

def get_calibration_curve(data_points):
    
    BLM_Cal_x_time = np.array([0., 3., 5., 7., 9.])
    # mVs/proton
    # BLM_Cal_y = np.array([2.22E-16, 2.59E-16, 4.31E-15, 1.60E-14, 3.50E-14])
    # vS/proton
    BLM_Cal_y = np.array([2.22E-16, 2.59E-16, 4.31E-15, 1.60E-14, 3.50E-14])
    
    a_s = divided_diff(BLM_Cal_x_time, BLM_Cal_y)[0, :]
    time_array = np.linspace(0., 10., data_points)
    
    # Original 1F method
    return newton_poly(a_s, BLM_Cal_x_time, time_array)   
    

def calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200, max_E=800):
    
    BLM_Cal_x_time = np.array([0., 3., 5., 7., 9.])
    # mVs/proton
    # BLM_Cal_y = np.array([2.22E-16, 2.59E-16, 4.31E-15, 1.60E-14, 3.50E-14])
    # vS/proton
    BLM_Cal_y = np.array([2.22E-16, 2.59E-16, 4.31E-15, 1.60E-14, 3.50E-14])
    
    
    time_array = np.linspace(t_min, t_max, data_points)
    
    # Default if in storage ring mode (fixed 70 MeV)
    if max_E == 70: return np.ones(len(time_array))*BLM_Cal_y[0]
    
    # calculate data points to pass to calibration curve (only returns between 0 - 10 ms)
    t_range = t_max - t_min
    # t_excess = t_range - 10.
    t_scale = data_points / t_range # points per millisecond
    #print(t_scale)
    cal_data_points = int(t_scale * 10)    
    pre_data_points = int(t_scale * -t_min)
    post_data_points = int(t_scale * (t_max-10))
    
    assert( (cal_data_points + pre_data_points + post_data_points) == data_points)
    
    a_s = divided_diff(BLM_Cal_x_time, BLM_Cal_y)[0, :]
    
    # Original 1F method
    calibration_curve = get_calibration_curve(cal_data_points) # newton_poly(a_s, BLM_Cal_x_time, time_array)
        
    # Conditional: if time before first calibration data point, fix value
    if np.any(time_array < BLM_Cal_x_time[0]): 
        #calibration_curve[np.where(time_array < 0)] = BLM_Cal_y[0]
        calibration_curve = np.concatenate([np.ones(pre_data_points)*BLM_Cal_y[0], calibration_curve])
    
    # Find Maximum - dependent on energy
    cal_max = 4.63E-14 # default to interpolated 800 MeV value

    # Conditional: if time before first calibration data point, fix value
    if np.any(time_array > 10.): 
        #calibration_curve[np.where(time_array < 0)] = BLM_Cal_y[0]
        calibration_curve = np.concatenate([calibration_curve, np.ones(post_data_points)*cal_max ])
    
    # Conditional: between first two points use linear interpolation
    
    #calibration_curve[np.where(time_array < 0)] = BLM_Cal_y[0]
        
        
    return calibration_curve

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"BLM Calibration Factor $K_{BLM}$ $(\frac{Vs}{proton})$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)

ax1.scatter(BLM_Cal_x_time,BLM_Cal_y, marker='X', c='r', label='Data')
#ax1.plot(np.linspace(-0.5, 10.5, 2200), np.interp(np.linspace(-0.5, 10.5, 2200), BLM_Cal_x_time, BLM_Cal_y))
ax1.plot(x_data, calibration_curve_beta(), label='Interpolated')
ax1.plot(x_data, calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200, max_E=70), label='70 MeV')

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)
ax1.legend()

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Interpolation with modified 1F method');

BLM_data = input_data_cycle[6]

blm_sum_list = []
n_min = 0
n_max = 38
for i in np.linspace(n_min, n_max, int(n_max - n_min), dtype=int):
    blm_sum_list.append(input_data_cycle[i])
    
BLM_sum = np.sum(blm_sum_list, axis=0) 

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"$I_{BLM}~(V)$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)

ax1.plot(x_data, BLM_sum)

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('BLM data / Calibration Curve (WRONG)');

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"$\frac{I_{BLM}}{K_{BLM}}$ $(\frac{V \cdot proton}{V \cdot s} = \frac{proton}{s})$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)

ax1.plot(x_data, BLM_data/calibration_curve_beta())

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('BLM data / Calibration Curve (WRONG)');

def find_diff_between_adj(array):
    diff_integral = np.diff(array)
    diff_integral = np.insert(diff_integral,0,array[0])
    
    return diff_integral

def get_integral_updated(x, array):
    in_array = get_integral(x, array)
    
    out_array = []
    for i in range(1, len(in_array)):
        out_array.append((in_array[i]-in_array[i-1]))
    
    return np.insert(out_array,0,in_array[0])
    #out_array = np.insert(out_array,0,in_array[0])
    #return out_array

def get_integral(x, value):
    integral = scipy.integrate.cumulative_trapezoid(value, x=x)
    
    return integral

integrated_BLM_data = (get_integral(np.linspace(-0.5, 10.5, 2200), BLM_sum))

Eshers_integrated_BLM_data = get_integral_updated(np.linspace(-0.5, 10.5, 2200), BLM_sum)

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax3 = ax1.twinx()
ax1.set_ylabel(r"$\int I_{BLM}~dt~(V \cdot s)$")
ax3.set_ylabel(r"$I_{BLM}~(V)$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)

ax1.plot(x_data[:-1], integrated_BLM_data, c='b', label='1F Integration')
ax1.plot(x_data[:-1], Eshers_integrated_BLM_data, c='r', lw=0.5, label='Esher Integration')
ax1.set_ylim(-0.0005, 0.003)
ax3.plot(np.linspace(-0.5, 10.5, 2200),BLM_sum, c='g', label='BLM Sum')
ax3.set_ylim(-0.3, 0.8)
ax1.legend(loc=5)
ax3.legend(loc=6)


ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Integrated BLM Sum');

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
#ax3 = ax1.twinx()
ax1.set_ylabel(r"$\frac{\int I_{BLM} dt}{K_{BLM}}$ $(\frac{V \cdot proton}{V \cdot s} = proton)$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)
x_data2 = np.linspace(-0.5, 10.5, data_points-1)

ax1.plot(x_data[:-1], integrated_BLM_data/(calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200)[:-1]))
ax1.plot(x_data[:-1], Eshers_integrated_BLM_data/(calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200)[:-1]), c='r', lw=0.5)
#ax1.plot(x_data[:-1], get_integral_updated(x_data,BLM_data)/calibration_curve_beta(2200))

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Integrated BLM data / Calibration Curve = PROTONS');

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax1.set_ylabel(r"Synchrotron Beam Intensity $I_{R5IM}$ $(protons)$")

ax1.plot(np.linspace(-0.5, 10.5, 2200), R5IM_data_cycle_protons)
ax1.set_xlim(-0.5, 10.5)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.0f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(np.linspace(0., 10., 11))
ax2.set_xticklabels(tick_function(np.linspace(0., 10., 11)))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('R5IM Intensity Data');

integration_time_factor = 1E-3

proton_data = integrated_BLM_data/(calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200)[:-1])

Eshers_proton_data = Eshers_integrated_BLM_data/(calibration_curve_beta(t_min=-0.5, t_max=10.5, data_points=2200)[:-1])

proton_data_updated = proton_data * integration_time_factor

Eshers_proton_data_updated = Eshers_proton_data * integration_time_factor
    
fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
#ax3 = ax1.twinx()
ax1.set_ylabel(r"$\frac{\int I_{BLM} dt}{K_{BLM}}$ $(\frac{V \cdot proton}{V \cdot s} = proton)$")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)
x_data2 = np.linspace(-0.5, 10.5, data_points-1)

ax1.plot(x_data[:-1], proton_data_updated)
ax1.plot(x_data[:-1], Eshers_proton_data_updated, c='r', lw=0.5)
#ax1.plot(x_data[:-1], get_integral_updated(x_data,BLM_data)/calibration_curve_beta(2200))

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Integrated BLM data / Calibration Curve = PROTONS');

time_array = np.linspace(-0.5, 10.5, 2200)

E = synchrotron_kinetic_energy(800., time_array)

joules_data = proton_data_updated * (E[:-1] + 938.27208816E6) * 1.602E-19

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax2 = ax1.twiny()
ax3 = ax1.twinx()
ax1.set_ylabel(r"$\frac{\int I_{BLM} dt}{K_{BLM}}$ $(\frac{V \cdot proton}{V \cdot s} = proton)$")
ax3.set_ylabel(r"Lost Beam Energy (Joules)")

data_points = 2200
x_data = np.linspace(-0.5, 10.5, data_points)
x_data2 = np.linspace(-0.5, 10.5, data_points-1)

ax1.plot(x_data[:-1], proton_data_updated)
ax3.plot(x_data[:-1], joules_data, c='r', lw=0.5)
#ax1.plot(x_data[:-1], get_integral_updated(x_data,BLM_data)/calibration_curve_beta(2200))

ax1.set_xlim(-1, 11)
ax1.set_xlabel(r"Synchrotron: cycle time $t_{cycle}$ $(ms)$")
ax1.grid(which='both', ls=':', lw=0.5)

def tick_function(time):
    E = synchrotron_kinetic_energy(800., time)*1E-6
    return ["%.2f" % z for z in E]

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(BLM_Cal_x_time)
ax2.set_xticklabels(tick_function(BLM_Cal_x_time))
ax2.set_xlabel(r"Synchrotron: Kinetic Energy $E_{Kinetic}$ $(MeV)$");
plt.title('Integrated BLM data / Calibration Curve = PROTONS');

integrated_BLM_test = np.abs(find_diff_between_adj(get_integral(np.linspace(-0.5, 10.5, 2200)[:-1], joules_data)))
    


NameError: name 'synchrotron_kinetic_energy' is not defined

In [246]:
# x axis is time interval, change time interval

def createX(start = -0.5, end = 10.5):
    x_data = np.linspace(start, end, nOfSamples)
    return x_data
    
def setX(start = -0.5, end = 10.5):
    filteredX = filter(lambda num: start <= num <= end, x_data)
    return (list(filteredX))

def clearX():
    x_data.clear()
    filteredX.clear()


In [225]:
# y axis is loss signal, must convert from v to p and c to j
# just create vY, pY, cY, jY

def V2P(data):
    v = [i * 4E12 for i in data]
    
def V2C(data):
    c = []
    
def V2J(data):
    j = []

vY = [] # whatever the raw voltage is
pY = V2P(vY)
cY = V2C(vY)
jY = V2J(vY)

NameError: name 'V2P' is not defined